In [1]:
import tensorflow as tf
print('Tensorflow version:', tf.__version__)

2025-11-20 21:59:30.772277: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 21:59:30.820046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 21:59:32.525910: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Tensorflow version: 2.20.0


In [2]:
# Load mnist dataset. This is a dataset of 60k 28x28 grayscale images of the 10 digits.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# The pixel values of each image ranges from 0 to 255. 
# This scales the values to a range of 0 to 1 by dividing each one by 255.0
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
# Build the Sequential model
# Sequential model is useful for stacking 
# layers where each layer has one input tensor and one output tensor.
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28,28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # The `Dropout` layer randomly sets input units to 0 with a frequency of
    # `rate` at each step during training time, which helps prevent overfitting.
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

I0000 00:00:1763693976.867905   32006 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
# For each example, the model returns a vector of logits or log-odds scores, one for 
# each class.
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.9927967 , -0.46119627,  0.08016752, -0.8921521 , -0.7084618 ,
         0.90742314,  0.67023677,  0.39673868,  0.6838846 ,  0.0387812 ]],
      dtype=float32)

In [5]:
# The tf.nn.softmax function convers these logits to probabilities for each class
tf.nn.softmax(predictions).numpy()

array([[0.03106617, 0.05286386, 0.09083855, 0.03435557, 0.04128315,
        0.20775081, 0.16388305, 0.12466808, 0.16613503, 0.0871558 ]],
      dtype=float32)

In [6]:
# Define the loss function
# This function takes a vector of ground truth values and a vector of logits and
# returns a scalar loss for each example.
lossfn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
# The untrained model gives probabilities close to random (so 1/10 for reach class,
# since there are 10 digits). So the initial loss should be close to 
# -tf.math.log(1/10) ~= 2.3
lossfn(y_train[:1], predictions).numpy()

np.float32(1.5714159)

In [8]:
# Before training, we must compile the model
model.compile(optimizer='adam', loss=lossfn, metrics=['accuracy'])

In [9]:
# Preview the model details with model.summary()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Use Model.fit to adjust the model prams and minimize loss
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2025-11-20 21:59:52.298439: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fe2100045b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-20 21:59:52.298475: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-11-20 21:59:52.335199: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-20 21:59:52.428894: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600
2025-11-20 21:59:52.495148: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-20 21:59:53.

  60/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5068 - loss: 1.6464

I0000 00:00:1763693994.737073   32072 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9153 - loss: 0.2945
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9573 - loss: 0.1444
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9682 - loss: 0.1071
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9735 - loss: 0.0873
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9773 - loss: 0.0727


In [11]:
# We can now evaluate the model
model.evaluate(x_test, y_test, verbose=2)

2025-11-20 22:00:52.587637: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 20 bytes spill stores, 20 bytes spill loads



313/313 - 2s - 6ms/step - accuracy: 0.9765 - loss: 0.0731


[0.07314564287662506, 0.9764999747276306]